In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install python-dotenv
!pip install pyngrok
from pyngrok import ngrok, conf

# ngrok 토큰 설정 (토큰을 발급받은 값으로 교체하세요)
NGROK_AUTH_TOKEN = "2oxetJl511S59IGSF21JV5t6CWk_6ona4iFbtYo5AbAM6Xexr"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

print("ngrok 토큰이 설정되었습니다.")

ngrok 토큰이 설정되었습니다.


In [ ]:
### Final App ###

# ===== 1. 기본 설정 =====
# Flask 웹 애플리케이션의 기본 설정 및 필요한 라이브러리 임포트

# ----- 필수 라이브러리 임포트 -----
# 웹 프레임워크 관련
from flask import Flask, render_template, request, jsonify, Response  # Flask 웹 프레임워크 핵심 컴포넌트
import json  # JSON 데이터 처리용

# 서버 및 데이터베이스
from pyngrok import ngrok  # 로컬 서버 터널링
import psycopg2  # PostgreSQL 데이터베이스 연결
import pandas as pd  # 데이터 처리 및 분석
from dotenv import load_dotenv # 환경변수 로드

# 딥러닝 관련
import torch  # PyTorch 딥러닝 프레임워크
import torch.nn as nn  # 신경망 모듈
import torch.nn.functional as F  # 신경망 함수
from transformers import AutoModel, AutoTokenizer  # 사전 학습된 모델 및 토크나이저

# 유틸리티
import os  # 파일 및 디렉토리 조작
import re  # 정규 표현식 처리
import pickle  # 객체 직렬화
import traceback  # 상세 에러 추적
import threading  # 멀티스레딩
import time  # 시간 관련 기능
import os 
from pathlib import Path

# ----- GPU 설정 -----
# CUDA GPU 사용 가능시 GPU 사용, 아닐 경우 CPU 사용
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ----- 환경변수 경로 설정 및 불러오기 -----
current_path = Path().absolute()
env_path = current_path / '.env'
load_dotenv(env_path)

# ----- Flask 앱 초기화 -----
app = Flask(
    __name__,
    # 템플릿 파일 경로 설정 (HTML 파일 위치)
    template_folder="/content/drive/MyDrive/Real_Life_Application_AI/Term_Project/App/templates",
)

# ----- Flask 앱 설정 -----
app.config['TIMEOUT'] = 600  # 요청 타임아웃 10분 설정

# ===== 2. 전역 변수 =====
progress = {}  # 진행률 저장
recommendations_cache = {} # 추천결과 저장

# ===== 3. 데이터베이스 관련 함수 =====

def fetch_data_from_rds():
    """
    RDS(Amazon Relational Database Service)에서 도서 데이터를 가져오는 함수

    Returns:
        pandas.DataFrame: 전처리된 도서 데이터
    """
    # PostgreSQL 데이터베이스 연결 설정
    conn = psycopg2.connect(
        host=os.getenv('DB_HOST'),
        database=os.getenv('DB_NAME'),
        user=os.getenv('DB_USER'),
        password=os.getenv('DB_PASSWORD'),
        port=os.getenv('DB_PORT')
    )
    # 학습용 도서 데이터 조회
    query = "SELECT * FROM BOOK_CB_4_T;"
    df = pd.read_sql(query, conn)
    df.columns = df.columns.str.upper()  # 열 이름 대문자로 통일
    conn.close()
    df = clean_book_data(df)  # 데이터 전처리 수행
    return df

def save_user_preference(nickname, book_isbn, preference):
    """
    사용자의 도서 선호도를 데이터베이스에 저장하는 함수

    Args:
        nickname (str): 사용자 닉네임
        book_isbn (str): 도서 ISBN
        preference (str): 선호도 ('like' 또는 'dislike')
    """
    try:
        # 데이터베이스 연결
        conn = psycopg2.connect(
            host=os.getenv('DB_HOST'),
            database=os.getenv('DB_NAME'),
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASSWORD'),
            port=os.getenv('DB_PORT')
        )
        cursor = conn.cursor()

        # UPSERT 쿼리 실행 (INSERT + UPDATE)
        query = """
            INSERT INTO user_preferences (nickname, isbn, preference)
            VALUES (%s, %s, %s)
            ON CONFLICT (nickname, isbn)
            DO UPDATE SET preference = EXCLUDED.preference;
        """
        cursor.execute(query, (nickname, book_isbn, preference))
        conn.commit()
        conn.close()
    except Exception as e:
        print(f"Error saving preference: {str(e)}")

def load_user_preferences(nickname):
    """
    사용자의 도서 선호도 정보를 데이터베이스에서 가져오는 함수

    Args:
        nickname (str): 사용자 닉네임

    Returns:
        dict: ISBN을 키로 하고 선호도를 값으로 하는 딕셔너리
    """
    try:
        # 데이터베이스 연결
        conn = psycopg2.connect(
            host=os.getenv('DB_HOST'),
            database=os.getenv('DB_NAME'),
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASSWORD'),
            port=os.getenv('DB_PORT')
        )
        # 사용자별 선호도 데이터 조회
        query = "SELECT isbn, preference FROM user_preferences WHERE nickname = %s;"
        df = pd.read_sql(query, conn, params=(nickname,))
        conn.close()

        return df.set_index('isbn')['preference'].to_dict()
    except Exception as e:
        print(f"Error loading preferences: {str(e)}")
        return {}

def save_user_embedding(nickname, embedding_tensor):
    """
    사용자의 임베딩 벡터를 데이터베이스에 저장하는 함수

    Args:
        nickname (str): 사용자 닉네임
        embedding_tensor (torch.Tensor): 사용자의 도서 취향을 나타내는 임베딩 벡터
    """
    try:
        # PyTorch 텐서를 NumPy 배열로 변환 후 바이너리 직렬화
        embedding_binary = pickle.dumps(embedding_tensor.cpu().numpy())

        # 데이터베이스 연결
        conn = psycopg2.connect(
            host=os.getenv('DB_HOST'),
            database=os.getenv('DB_NAME'),
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASSWORD'),
            port=os.getenv('DB_PORT')
        )
        cursor = conn.cursor()

        # UPSERT 쿼리 실행 및 타임스탬프 업데이트
        query = """
            INSERT INTO user_embeddings (nickname, embedding)
            VALUES (%s, %s)
            ON CONFLICT (nickname)
            DO UPDATE SET
                embedding = EXCLUDED.embedding,
                updated_at = CURRENT_TIMESTAMP;
        """
        cursor.execute(query, (nickname, embedding_binary))
        conn.commit()
        conn.close()
    except Exception as e:
        print(f"Error saving embedding: {str(e)}")

def load_user_embedding(nickname):
    """
    사용자의 임베딩 벡터를 데이터베이스에서 가져오는 함수

    Args:
        nickname (str): 사용자 닉네임

    Returns:
        torch.Tensor: 사용자의 임베딩 벡터, 없으면 None 반환
    """
    try:
        # 데이터베이스 연결
        conn = psycopg2.connect(
            host=os.getenv('DB_HOST'),
            database=os.getenv('DB_NAME'),
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASSWORD'),
            port=os.getenv('DB_PORT')
        )
        cursor = conn.cursor()

        # 임베딩 데이터 조회
        query = "SELECT embedding FROM user_embeddings WHERE nickname = %s;"
        cursor.execute(query, (nickname,))
        result = cursor.fetchone()
        conn.close()

        if result:
            # 바이너리 데이터를 PyTorch 텐서로 변환
            embedding_array = pickle.loads(result[0])
            return torch.from_numpy(embedding_array)
        return None
    except Exception as e:
        print(f"Error loading embedding: {str(e)}")
        return None

def delete_user_preferences(nickname):
    """
    사용자의 선호도 정보를 데이터베이스에서 삭제하는 함수

    Args:
        nickname (str): 사용자 닉네임

    Returns:
        bool: 삭제 성공 여부
    """
    try:
        # 데이터베이스 연결
        conn = psycopg2.connect(
            host=os.getenv('DB_HOST'),
            database=os.getenv('DB_NAME'),
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASSWORD'),
            port=os.getenv('DB_PORT')
        )
        cursor = conn.cursor()

        # 선호도 데이터 삭제
        query = "DELETE FROM user_preferences WHERE nickname = %s;"
        cursor.execute(query, (nickname,))
        conn.commit()
        cursor.close()
        conn.close()
        print(f"Deleted preferences for user: {nickname}")
        return True
    except Exception as e:
        print(f"Error deleting user preferences: {str(e)}")
        return False

def delete_user_embedding(nickname):
    """
    사용자의 임베딩 데이터를 데이터베이스에서 삭제하는 함수

    Args:
        nickname (str): 사용자 닉네임

    Returns:
        bool: 삭제 성공 여부
    """
    try:
        # 데이터베이스 연결
        conn = psycopg2.connect(
            host=os.getenv('DB_HOST'),
            database=os.getenv('DB_NAME'),
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASSWORD'),
            port=os.getenv('DB_PORT')
        )
        cursor = conn.cursor()

        # 임베딩 데이터 삭제
        query = "DELETE FROM user_embeddings WHERE nickname = %s;"
        cursor.execute(query, (nickname,))
        conn.commit()
        cursor.close()
        conn.close()
        print(f"Deleted embedding for user: {nickname}")
        return True
    except Exception as e:
        print(f"Error deleting user embedding: {str(e)}")
        return False

# ===== 4. 데이터 전처리 함수 =====
def clean_text(text):
   """텍스트 전처리 함수"""
   if pd.isna(text):
       return ""

   # 문자열 변환
   text = str(text)

   # 불필요한 공백 제거
   text = ' '.join(text.split())

   # 특수문자 처리 (일부 기본적인 문장부호 유지)
   text = re.sub(r'[^가-힣a-zA-Z0-9\s\.,!?]', '', text)

   return text

def clean_book_data(df):
   """데이터프레임 내 모든 문자열 열에서 쌍따옴표 제거."""
   for column in df.select_dtypes(include=['object']).columns:
       df[column] = df[column].str.strip('"')
   return df

# ===== 5. 모델 관련 클래스 및 함수 =====

class ImprovedBookClassifier(nn.Module):
    """
    개선된 도서 분류 모델 클래스
    BERT 기반의 멀티헤드 어텐션과 계층적 분류기를 결합한 신경망 모델

    특징:
        - BERT를 통한 텍스트 임베딩
        - 멀티헤드 어텐션으로 중요 특성 강조
        - 계층적 분류기로 점진적 특성 추출
        - Dropout과 LayerNorm을 통한 정규화

    Args:
        model_name (str): 사전학습된 BERT 모델명 (예: 'skt/kobert-base-v1')
        num_labels (int): 출력 클래스 수
        dropout_rate (float): 드롭아웃 비율 (기본값: 0.3)
    """
    def __init__(self, model_name, num_labels, dropout_rate=0.3):
        super().__init__()
        # BERT 모델 초기화
        self.bert = AutoModel.from_pretrained(model_name)

        # 계층적 분류기 정의
        self.classifier = nn.Sequential(
            # 첫 번째 레이어: 차원 축소 및 특성 추출 (2048 -> 1024)
            nn.Linear(self.bert.config.hidden_size * 2, 1024),
            nn.LayerNorm(1024),  # 배치 정규화
            nn.ReLU(),  # 비선형성 추가
            nn.Dropout(dropout_rate),  # 과적합 방지

            # 두 번째 레이어: 중간 특성 학습 (1024 -> 512)
            nn.Linear(1024, 512),
            nn.LayerNorm(512),
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            # 세 번째 레이어: 고수준 특성 추출 (512 -> 256)
            nn.Linear(512, 256),
            nn.LayerNorm(256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),

            # 출력 레이어: 최종 분류 (256 -> num_labels)
            nn.Linear(256, num_labels)
        )

        # 멀티헤드 어텐션 레이어
        self.multihead_attention = nn.MultiheadAttention(
            embed_dim=self.bert.config.hidden_size,  # BERT 임베딩 차원
            num_heads=8,  # 어텐션 헤드 수
            dropout=dropout_rate  # 드롭아웃 비율
        )

    def forward(self, input_ids, attention_mask):
        """
        순전파 함수: 입력 텍스트를 처리하여 분류 결과 출력

        프로세스:
        1. BERT를 통한 텍스트 인코딩
        2. 멀티헤드 어텐션 적용
        3. 최대 풀링과 평균 풀링 결합
        4. 계층적 분류기를 통한 최종 예측

        Args:
            input_ids (torch.Tensor): 입력 토큰 ID [batch_size, seq_length]
            attention_mask (torch.Tensor): 어텐션 마스크 [batch_size, seq_length]

        Returns:
            torch.Tensor: 분류 로짓 [batch_size, num_labels]
        """
        # BERT 모델을 통한 텍스트 인코딩
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        sequence_output = outputs.last_hidden_state  # [batch_size, seq_length, hidden_size]

        # 멀티헤드 어텐션 적용
        # 시퀀스를 [seq_length, batch_size, hidden_size] 형태로 변환
        attention_output, _ = self.multihead_attention(
            sequence_output.transpose(0, 1),
            sequence_output.transpose(0, 1),
            sequence_output.transpose(0, 1),
            key_padding_mask=~attention_mask.bool()
        )
        attention_output = attention_output.transpose(0, 1)  # 원래 형태로 복원

        # 풀링 레이어: 최대값과 평균값 결합
        max_pooled = torch.max(attention_output, dim=1)[0]  # [batch_size, hidden_size]
        avg_pooled = torch.mean(attention_output, dim=1)  # [batch_size, hidden_size]
        pooled_output = torch.cat([max_pooled, avg_pooled], dim=1)  # [batch_size, hidden_size*2]

        # 분류기를 통한 최종 예측
        return self.classifier(pooled_output)

# 모델 설정 및 초기화
num_labels = 4  # 대분류 카테고리 수
checkpoint_path = '/content/drive/MyDrive/Real_Life_Application_AI/Term_Project/App/KoBERT/best_model.pt'

# 모델 인스턴스 생성 및 GPU 할당
model = ImprovedBookClassifier('skt/kobert-base-v1', num_labels=num_labels)
model = model.to(device)
print(f"Model initialized on device: {next(model.parameters()).device}")

# 체크포인트에서 학습된 가중치 로드
checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)
model.eval()  # 평가 모드로 설정

# BERT 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1")

def get_book_embeddings(titles, authors, stories, process_name=""):
    """
    도서 정보로부터 임베딩 벡터를 생성하는 함수

    프로세스:
    1. 입력 데이터 배치 처리
    2. 텍스트 전처리 및 토큰화
    3. 모델을 통한 임베딩 생성
    4. 배치별 임베딩 결합

    Args:
        titles (list): 도서 제목 리스트
        authors (list): 작가 이름 리스트
        stories (list): 줄거리 리스트
        process_name (str): 처리 과정 식별자 (로깅용)

    Returns:
        torch.Tensor: 도서 임베딩 벡터 [num_books, embedding_dim]

    Raises:
        ValueError: 임베딩 생성 실패 시
    """
    print(f"\nStarting embedding generation for {process_name}")
    print(f"Total books to process: {len(titles)}")

    embeddings = []
    batch_size = 16  # 배치 크기

    model.eval()  # 평가 모드 설정
    print(f"Model device: {next(model.parameters()).device}")

    # 배치 단위로 처리
    for idx in range(0, len(titles), batch_size):
        print(f"Processing batch {idx//batch_size + 1}/{(len(titles)-1)//batch_size + 1}")
        batch_titles = titles[idx:idx + batch_size]
        batch_authors = authors[idx:idx + batch_size]
        batch_stories = stories[idx:idx + batch_size]

        try:
            # 텍스트 전처리 및 결합
            batch_texts = []
            for title, author, story in zip(batch_titles, batch_authors, batch_stories):
                title = clean_text(title) if pd.notna(title) else ""
                author = clean_text(author) if pd.notna(author) else ""
                story = clean_text(story) if pd.notna(story) else ""
                text = f"[제목] {title} [작가] {author} [줄거리] {story}"
                batch_texts.append(text)

            # 토큰화
            inputs = tokenizer(
                batch_texts,
                padding=True,  # 패딩 적용
                truncation=True,  # 최대 길이 초과 시 자르기
                max_length=512,  # 최대 시퀀스 길이
                return_tensors="pt"  # PyTorch 텐서 반환
            )

            # GPU로 입력 데이터 이동
            input_dict = {
                'input_ids': inputs['input_ids'].to(device),
                'attention_mask': inputs['attention_mask'].to(device)
            }

            # 임베딩 생성 (그래디언트 계산 없이)
            with torch.no_grad():
                model.to(device)
                outputs = model(**input_dict)
                if isinstance(outputs, torch.Tensor):
                    batch_embeddings = outputs.to('cpu')
                else:
                    batch_embeddings = outputs
                embeddings.append(batch_embeddings)
                print(f"Successfully processed batch {idx//batch_size + 1}")

        except Exception as e:
            print(f"Error processing batch {idx//batch_size + 1}: {str(e)}")
            print(f"Device info - Model: {next(model.parameters()).device}")
            for k, v in input_dict.items():
                print(f"{k} device: {v.device}")
            continue

    # 임베딩 생성 확인
    if not embeddings:
        raise ValueError(f"No embeddings were generated for {process_name}")

    try:
        # 모든 배치의 임베딩을 CPU로 이동 후 결합
        embeddings = [e.cpu() if isinstance(e, torch.Tensor) else e for e in embeddings]
        final_embeddings = torch.cat(embeddings, dim=0)
        print(f"Completed embedding generation for {process_name}")
        print(f"Generated embeddings shape: {final_embeddings.shape}\n")
        return final_embeddings

    except Exception as e:
        print(f"Error during final concatenation: {str(e)}")
        raise

# ===== 6. 추천 시스템 핵심 함수 =====

def update_recommendations(nickname, book_isbn, preference):
    """
    사용자의 선호도 입력에 따라 실시간으로 추천 목록을 업데이트하는 함수

    동작 방식:
    1. 같은 카테고리의 도서들의 유사도 점수를 조정
    2. 좋아요: 유사도 20% 증가 (1.2배)
    3. 싫어요: 유사도 20% 감소 (0.8배)
    4. 수정된 유사도에 따라 추천 목록 재정렬

    Args:
        nickname (str): 사용자 닉네임
        book_isbn (str): 평가된 도서의 ISBN
        preference (str): 선호도 ('like' 또는 'dislike')
    """
    try:
        # 캐시된 추천 목록이 있는 경우에만 처리
        if nickname in recommendations_cache:
            recommendations = recommendations_cache[nickname]

            # 좋아요 처리: 같은 카테고리 도서들의 유사도 증가
            if preference == 'like':
                # 좋아요 받은 도서 찾기
                liked_book = next(book for book in recommendations
                                if book['ISBN_THIRTEEN_NO'] == book_isbn)
                category = liked_book['CATEGORY_B']

                # 같은 카테고리 도서들의 유사도 가중치 증가 (20% 증가)
                for book in recommendations:
                    if book['CATEGORY_B'] == category:
                        book['similarity'] *= 1.2

            # 싫어요 처리: 같은 카테고리 도서들의 유사도 감소
            elif preference == 'dislike':
                # 싫어요 받은 도서 찾기
                disliked_book = next(book for book in recommendations
                                   if book['ISBN_THIRTEEN_NO'] == book_isbn)
                category = disliked_book['CATEGORY_B']

                # 같은 카테고리 도서들의 유사도 가중치 감소 (20% 감소)
                for book in recommendations:
                    if book['CATEGORY_B'] == category:
                        book['similarity'] *= 0.8

            # 수정된 유사도에 따라 추천 목록 재정렬
            recommendations.sort(key=lambda x: x['similarity'], reverse=True)
            recommendations_cache[nickname] = recommendations

            print(f"Updated recommendations for {nickname}")
    except Exception as e:
        print(f"Error updating recommendations: {str(e)}")

def analyze_user_preferences(nickname):
    """
    사용자의 도서 선호도를 종합적으로 분석하는 함수

    분석 항목:
    1. 전체 카테고리 분포
    2. 선호하는 카테고리 (선호도 점수 > 0.3)
    3. 선호하지 않는 카테고리 (선호도 점수 < -0.3)
    4. 카테고리별 선호도 점수

    선호도 점수 계산 방식:
    score = (좋아요 수 - 싫어요 수) / (전체 평가 수)
    범위: -1.0 ~ 1.0

    Args:
        nickname (str): 분석할 사용자의 닉네임

    Returns:
        dict: 분석 결과를 담은 딕셔너리
            - overall_distribution: 카테고리별 도서 분포 (%)
            - favorite_categories: 선호하는 카테고리 목록
            - disliked_categories: 선호하지 않는 카테고리 목록
            - preference_scores: 카테고리별 선호도 점수
    """
    try:
        print(f"Starting analysis for user: {nickname}")

        # 데이터베이스 연결
        conn = psycopg2.connect(
            host=os.getenv('DB_HOST'),
            database=os.getenv('DB_NAME'),
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASSWORD'),
            port=os.getenv('DB_PORT')
        )

        # 사용자 선호도 데이터 조회 (큰따옴표 제거 포함)
        query = """
            SELECT
                up.isbn,
                up.preference,
                TRIM(BOTH '"' FROM b.category_b) as category_b
            FROM user_preferences up
            JOIN book_cb_4 b ON up.isbn = TRIM(BOTH '"' FROM b.isbn_thirteen_no)
            WHERE up.nickname = %s;
        """

        print(f"Executing query for nickname: {nickname}")
        preferences_df = pd.read_sql(query, conn, params=(nickname,))
        print(f"Retrieved {len(preferences_df)} preference records")

        # 선호도 데이터가 없는 경우 기본값 반환
        if len(preferences_df) == 0:
            print("No preferences found for user")
            return {
                'overall_distribution': {},
                'favorite_categories': ['아직 선호하는 카테고리가 없습니다.'],
                'disliked_categories': ['아직 선호하지 않는 카테고리가 없습니다.'],
                'preference_scores': {}
            }

        # 1. 전체 카테고리 분포 분석
        category_distribution = preferences_df['category_b'].value_counts()
        # 백분율로 변환하고 소수점 2자리까지 반올림
        category_percentage = (category_distribution / len(preferences_df) * 100).round(2)

        # 2. 카테고리별 좋아요/싫어요 분석
        liked_categories = preferences_df[preferences_df['preference'] == 'like']['category_b'].value_counts()
        disliked_categories = preferences_df[preferences_df['preference'] == 'dislike']['category_b'].value_counts()

        # 3. 카테고리별 선호도 점수 계산
        preference_scores = {}
        for category in set(preferences_df['category_b']):
            likes = liked_categories.get(category, 0)  # 좋아요 수
            dislikes = disliked_categories.get(category, 0)  # 싫어요 수
            total = float(likes + dislikes)  # 전체 평가 수
            if total > 0:
                # 선호도 점수 = (좋아요 - 싫어요) / 전체
                score = (likes - dislikes) / total
                preference_scores[category] = round(score, 2)

        # 4. 선호/비선호 카테고리 선별
        # 선호도 점수 0.3 초과인 카테고리
        favorite_categories = [
            cat for cat, score in preference_scores.items()
            if score > 0.3
        ]

        # 선호도 점수 -0.3 미만인 카테고리
        disliked_categories = [
            cat for cat, score in preference_scores.items()
            if score < -0.3
        ]

        # 5. 결과 메시지 설정
        # 선호 카테고리가 없는 경우
        if not favorite_categories:
            if len(preferences_df[preferences_df['preference'] == 'like']) > 0:
                favorite_categories = ['아직 뚜렷한 선호 카테고리가 없습니다.']
            else:
                favorite_categories = ['아직 선호하는 카테고리가 없습니다.']

        # 비선호 카테고리가 없는 경우
        if not disliked_categories:
            if len(preferences_df[preferences_df['preference'] == 'dislike']) > 0:
                disliked_categories = ['아직 뚜렷한 비선호 카테고리가 없습니다.']
            else:
                disliked_categories = ['아직 선호하지 않는 카테고리가 없습니다.']

        # 6. 최종 분석 결과 생성
        analysis_result = {
            'overall_distribution': category_percentage.to_dict(),
            'favorite_categories': favorite_categories,
            'disliked_categories': disliked_categories,
            'preference_scores': preference_scores
        }

        print("Analysis result:", analysis_result)
        conn.close()
        return analysis_result

    except Exception as e:
        # 에러 발생 시 상세 로그 출력 및 기본값 반환
        print(f"Error in analyze_user_preferences: {str(e)}")
        import traceback
        print(traceback.format_exc())
        return {
            'overall_distribution': {},
            'favorite_categories': ['데이터를 불러오는 중 오류가 발생했습니다.'],
            'disliked_categories': ['데이터를 불러오는 중 오류가 발생했습니다.'],
            'preference_scores': {}
        }

# ===== 7. API 라우트 =====

@app.route('/')
def index():
    """메인 페이지 렌더링"""
    return render_template('index.html')

@app.route('/check_previous_preferences', methods=['POST'])
def check_previous_preferences():
    """
    사용자의 이전 선호도 기록을 확인하는 엔드포인트

    기능:
    1. 새로운 추천 요청 시 기존 데이터 삭제
    2. 기존 임베딩 존재 여부 확인

    Returns:
        JSON 응답:
        - has_preferences: 선호도 존재 여부
        - message: 사용자에게 표시할 메시지
    """
    try:
        nickname = request.form.get('nickname')
        action = request.form.get('action')  # 'new' 또는 undefined

        # 새로운 추천 시작 시 기존 데이터 삭제
        if action == 'new':
            delete_user_preferences(nickname)
            delete_user_embedding(nickname)
            if nickname in recommendations_cache:
                del recommendations_cache[nickname]
            return jsonify({'has_preferences': False})

        # 저장된 임베딩 확인
        user_embedding = load_user_embedding(nickname)
        if user_embedding is not None:
            return jsonify({
                'has_preferences': True,
                'message': '이전 추천 기록이 있습니다. 이전 추천을 확인하시겠습니까?'
            })
        else:
            return jsonify({'has_preferences': False})

    except Exception as e:
        print(f"Error checking preferences: {str(e)}")
        return jsonify({'error': str(e)}), 500

@app.route('/load_previous_recommendations', methods=['GET'])
def load_previous_recommendations():
    """
    이전 추천 결과를 로드하는 엔드포인트
    Server-Sent Events를 사용하여 실시간 진행 상황 전송

    프로세스:
    1. 캐시된 추천 확인
    2. 임베딩 기반 새로운 추천 생성
    3. 진행 상황 실시간 전송

    Returns:
        EventStream: 진행 상황 및 추천 결과
    """
    try:
        nickname = request.args.get('nickname')

        def generate_recommendations():
            """추천 생성 및 진행 상황 전송 제너레이터"""
            try:
                yield "data: {\"progress\": 0, \"status\": \"이전 추천 결과 불러오는 중...\"}\n\n"

                # 캐시된 추천 확인
                if nickname in recommendations_cache:
                    yield "data: {\"progress\": 90, \"status\": \"캐시된 결과 불러오는 중...\"}\n\n"
                    all_recommendations = recommendations_cache[nickname]
                else:
                    # 임베딩 로드
                    user_embedding = load_user_embedding(nickname)
                    if user_embedding is None:
                        raise ValueError("저장된 임베딩을 찾을 수 없습니다.")

                    # 도서 임베딩 생성
                    yield "data: {\"progress\": 30, \"status\": \"도서 특성 추출 중...\"}\n\n"
                    all_embeddings = get_book_embeddings(
                        book_data['TITLE_NM'].tolist(),
                        book_data['AUTHR_NM'].tolist(),
                        book_data['STORY'].tolist(),
                        "all books"
                    )

                    # 유사도 계산 및 추천 목록 생성
                    yield "data: {\"progress\": 60, \"status\": \"추천 도서 계산 중...\"}\n\n"
                    similarities = F.cosine_similarity(
                        all_embeddings,
                        user_embedding.unsqueeze(0)
                    )

                    book_data['similarity'] = similarities.cpu().numpy()
                    all_recommendations = (
                        book_data
                        .sort_values('similarity', ascending=False)
                        .head(100)
                        .to_dict('records')
                    )
                    recommendations_cache[nickname] = all_recommendations

                # 최종 결과 전송
                yield "data: {\"progress\": 100, \"status\": \"완료\", \"recommendations\": " + json.dumps(all_recommendations[:10]) + "}\n\n"

            except Exception as e:
                print(f"Error in generate_recommendations: {str(e)}")
                print(traceback.format_exc())
                error_data = {
                    "progress": 0,
                    "status": f"오류 발생: {str(e)}",
                    "error": str(e)
                }
                yield f"data: {json.dumps(error_data)}\n\n"

        return Response(
            generate_recommendations(),
            mimetype='text/event-stream',
            headers={
                'Cache-Control': 'no-cache',
                'Connection': 'keep-alive',
            }
        )

    except Exception as e:
        print(f"Error in load_previous_recommendations: {str(e)}")
        traceback.print_exc()
        return render_template('error.html', message=str(e))

@app.route('/select_books', methods=['POST'])
def select_books():
    """
    사용자가 선호하는 도서를 선택할 수 있는 페이지를 렌더링하는 엔드포인트

    기능:
    1. 카테고리별로 10권씩 도서 샘플링
    2. NaN 값 처리 및 데이터 정제

    Returns:
        str: 도서 선택 페이지 HTML
    """
    nickname = request.form['nickname']
    # 카테고리별로 10권씩 도서 샘플링
    sampled_books = book_data.groupby('CATEGORY_B').apply(
        lambda x: x.sample(10, replace=False) if len(x) >= 10 else x
    ).reset_index(drop=True)
    # NaN 값을 None으로 변환하여 JSON 직렬화 가능하게 함
    books = sampled_books.where(pd.notnull(sampled_books), None).to_dict(orient="records")
    return render_template('select_books.html', nickname=nickname, books=books)

@app.route('/recommend_books', methods=['POST', 'GET'])
def recommend_books():
    """
    도서 추천을 처리하는 엔드포인트
    GET: 실시간 추천 프로세스 실행
    POST: 추천 결과 페이지 렌더링

    주요 기능:
    1. 선택된 도서 기반 사용자 프로필 생성
    2. 전체 도서와의 유사도 계산
    3. 이전 선호도 반영
    4. 실시간 진행 상황 전송
    """
    try:
        if request.method == 'GET':
            books = request.args.get('books').split(',')
            nickname = request.args.get('nickname')

            def keep_alive():
                """연결 유지를 위한 하트비트 생성기"""
                while True:
                    yield "data: {\"progress\": -1, \"status\": \"keep-alive\"}\n\n"
                    time.sleep(5)

            def generate_recommendations():
                """추천 생성 및 진행 상황 전송 제너레이터"""
                try:
                    print(f"\nStarting recommendation process for {nickname}")
                    print(f"Selected books: {books}")

                    # 초기 연결 설정
                    yield "data: {\"progress\": 0, \"status\": \"연결 중...\"}\n\n"
                    progress[nickname] = 10
                    yield "data: {\"progress\": 10, \"status\": \"선택한 도서 분석 중...\"}\n\n"

                    # 선택된 도서 데이터 검색
                    selected_books = book_data[book_data['ISBN_THIRTEEN_NO'].isin(books)]
                    if len(selected_books) == 0:
                        raise ValueError("선택된 도서를 찾을 수 없습니다.")
                    print(f"Found {len(selected_books)} books in database")

                    # 선택된 도서의 임베딩 생성
                    progress[nickname] = 20
                    yield "data: {\"progress\": 20, \"status\": \"도서 특성 추출 중...\"}\n\n"
                    try:
                        selected_embeddings = get_book_embeddings(
                            selected_books['TITLE_NM'].tolist(),
                            selected_books['AUTHR_NM'].tolist(),
                            selected_books['STORY'].tolist(),
                            "selected books"
                        )
                    except Exception as e:
                        print(f"Error generating selected book embeddings: {str(e)}")
                        print(traceback.format_exc())
                        raise

                    # 전체 도서의 임베딩 생성
                    progress[nickname] = 50
                    yield "data: {\"progress\": 50, \"status\": \"사용자 취향 분석 중...\"}\n\n"
                    try:
                        all_embeddings = get_book_embeddings(
                            book_data['TITLE_NM'].tolist(),
                            book_data['AUTHR_NM'].tolist(),
                            book_data['STORY'].tolist(),
                            "all books"
                        )
                    except Exception as e:
                        print(f"Error generating all book embeddings: {str(e)}")
                        print(traceback.format_exc())
                        raise

                    # 추천 도서 선정
                    progress[nickname] = 80
                    yield "data: {\"progress\": 80, \"status\": \"맞춤 도서 선정 중...\"}\n\n"
                    try:
                        # 사용자 프로필 생성 및 저장
                        user_embedding = selected_embeddings.mean(dim=0)
                        save_user_embedding(nickname, user_embedding)

                        # 유사도 계산
                        similarities = F.cosine_similarity(
                            all_embeddings,
                            user_embedding.unsqueeze(0)
                        )

                        # 추천 도서 목록 생성
                        book_data['similarity'] = similarities.cpu().numpy()
                        recommendations = (
                            book_data
                            .sort_values('similarity', ascending=False)
                            .head(100)
                            .to_dict('records')
                        )

                        # 이전 선호도 반영
                        conn = psycopg2.connect(
                            host=os.getenv('DB_HOST'),
                            database=os.getenv('DB_NAME'),
                            user=os.getenv('DB_USER'),
                            password=os.getenv('DB_PASSWORD'),
                            port=os.getenv('DB_PORT')
                        )
                        query = "SELECT isbn, preference FROM user_preferences WHERE nickname = %s;"
                        preferences_df = pd.read_sql(query, conn, params=(nickname,))
                        conn.close()

                        # 선호도에 따른 유사도 조정
                        for _, row in preferences_df.iterrows():
                            isbn = row['isbn']
                            preference = row['preference']
                            book_category = next(
                                (book['category_b'] for book in recommendations
                                 if book['ISBN_THIRTEEN_NO'] == isbn),
                                None
                            )

                            if book_category:
                                for book in recommendations:
                                    if book['category_b'] == book_category:
                                        if preference == 'like':
                                            book['similarity'] *= 1.2
                                        elif preference == 'dislike':
                                            book['similarity'] *= 0.8

                        # 최종 추천 목록 정렬 및 캐시 저장
                        recommendations.sort(key=lambda x: x['similarity'], reverse=True)
                        recommendations_cache[nickname] = recommendations

                        # 최종 결과 전송
                        progress[nickname] = 100
                        final_data = {
                            "progress": 100,
                            "status": "완료",
                            "recommendations": recommendations[:10]
                        }
                        yield f"data: {json.dumps(final_data)}\n\n"

                    except Exception as e:
                        print(f"Error in processing recommendations: {str(e)}")
                        print(traceback.format_exc())
                        raise

                except Exception as e:
                    print(f"Critical error in generate_recommendations: {str(e)}")
                    print(traceback.format_exc())
                    error_data = {
                        "progress": 0,
                        "status": f"오류 발생: {str(e)}",
                        "error": str(e),
                        "traceback": traceback.format_exc()
                    }
                    yield f"data: {json.dumps(error_data)}\n\n"

            return Response(
                generate_recommendations(),
                mimetype='text/event-stream',
                headers={
                    'Cache-Control': 'no-cache',
                    'Connection': 'keep-alive',
                }
            )

        else:
            # POST 요청 처리 (페이지 렌더링)
            books = request.form.getlist('books')
            nickname = request.form['nickname']

            # 캐시된 추천 결과 사용
            if nickname in recommendations_cache:
                all_recommendations = recommendations_cache[nickname]
                return render_template(
                    'recommend_books.html',
                    nickname=nickname,
                    recommendations=all_recommendations[:10]
                )
            else:
                return render_template('error.html', message="추천 결과를 찾을 수 없습니다.")

    except Exception as e:
        print(f"Error in recommend_books: {str(e)}")
        return render_template('error.html', message=str(e))

@app.route('/update_preference', methods=['POST'])
def update_preference():
    """
    사용자의 도서 선호도를 업데이트하는 엔드포인트

    처리 과정:
    1. 선호도 데이터베이스 저장
    2. 사용자 임베딩 업데이트
    3. 추천 목록 재조정

    Returns:
        JSON 응답:
        - success: 업데이트 성공 여부
        - error: 실패 시 에러 메시지
    """
    try:
        # 요청 데이터 파싱
        data = request.json
        nickname = data.get('nickname')
        book_isbn = data.get('isbn')
        preference = data.get('preference')

        # 데이터베이스에 선호도 저장
        conn = psycopg2.connect(
            host=os.getenv('DB_HOST'),
            database=os.getenv('DB_NAME'),
            user=os.getenv('DB_USER'),
            password=os.getenv('DB_PASSWORD'),
            port=os.getenv('DB_PORT')
        )
        cursor = conn.cursor()

        # UPSERT 쿼리 실행
        query = """
            INSERT INTO user_preferences (nickname, isbn, preference)
            VALUES (%s, %s, %s)
            ON CONFLICT (nickname, isbn)
            DO UPDATE SET preference = EXCLUDED.preference;
        """
        cursor.execute(query, (nickname, book_isbn, preference))
        conn.commit()
        cursor.close()
        conn.close()

        # 임베딩 업데이트
        user_embedding = load_user_embedding(nickname)
        if user_embedding is not None:
            # 선호도가 업데이트된 도서의 임베딩 생성
            book_data_row = book_data[book_data['ISBN_THIRTEEN_NO'] == book_isbn].iloc[0]
            book_embedding = get_book_embeddings(
                [book_data_row['TITLE_NM']],
                [book_data_row['AUTHR_NM']],
                [book_data_row['STORY']]
            )[0]

            # 선호도에 따른 임베딩 조정
            if preference == 'like':
                # 좋아요: 기존 임베딩 80% + 새 도서 임베딩 20%
                user_embedding = (user_embedding * 0.8) + (book_embedding * 0.2)
            elif preference == 'dislike':
                # 싫어요: 기존 임베딩 80% - 새 도서 임베딩 20%
                user_embedding = (user_embedding * 0.8) - (book_embedding * 0.2)

            # 임베딩 정규화
            user_embedding = F.normalize(user_embedding.unsqueeze(0)).squeeze(0)

            # 업데이트된 임베딩 저장
            save_user_embedding(nickname, user_embedding)

            # 캐시된 추천 목록 업데이트
            if nickname in recommendations_cache:
                recommendations = recommendations_cache[nickname]

                # 선호도에 따른 유사도 조정
                if preference == 'like':
                    # 좋아요 받은 도서의 카테고리 찾기
                    liked_book = next(book for book in recommendations
                                    if book['ISBN_THIRTEEN_NO'] == book_isbn)
                    category = liked_book['CATEGORY_B']

                    # 같은 카테고리 도서들의 유사도 증가
                    for book in recommendations:
                        if book['CATEGORY_B'] == category:
                            book['similarity'] *= 1.2  # 20% 증가

                elif preference == 'dislike':
                    # 싫어요 받은 도서의 카테고리 찾기
                    disliked_book = next(book for book in recommendations
                                       if book['ISBN_THIRTEEN_NO'] == book_isbn)
                    category = disliked_book['CATEGORY_B']

                    # 같은 카테고리 도서들의 유사도 감소
                    for book in recommendations:
                        if book['CATEGORY_B'] == category:
                            book['similarity'] *= 0.8  # 20% 감소

                # 추천 목록 재정렬 및 캐시 업데이트
                recommendations.sort(key=lambda x: x['similarity'], reverse=True)
                recommendations_cache[nickname] = recommendations

        return jsonify({'success': True})

    except Exception as e:
        print(f"Error updating preference: {str(e)}")
        return jsonify({
            'success': False,
            'error': str(e)
        }), 500

@app.route('/load_more_books', methods=['GET'])
def load_more_books():
    """
    추가 추천 도서를 로드하는 엔드포인트
    무한 스크롤 구현을 위한 페이지네이션 처리

    프로세스:
    1. 기존 추천 목록 확인
    2. 필요시 새로운 추천 도서 생성
    3. 선호도 반영하여 목록 조정

    Returns:
        JSON 응답:
        - books: 추가 추천 도서 목록
        - has_more: 추가 도서 존재 여부
    """
    try:
        offset = int(request.args.get('offset', 0))
        nickname = request.args.get('nickname')
        print(f"Loading more books for {nickname} from offset {offset}")

        all_recommendations = recommendations_cache.get(nickname, [])

        # 남은 도서가 10개 미만일 때 추가 추천 생성
        if len(all_recommendations) - offset < 10:
            user_embedding = load_user_embedding(nickname)
            if user_embedding is not None:
                # 기존 추천 도서 ISBN 추출
                existing_isbns = set(book['ISBN_THIRTEEN_NO'] for book in all_recommendations)

                # 새로운 도서 임베딩 생성
                all_embeddings = get_book_embeddings(
                    book_data['TITLE_NM'].tolist(),
                    book_data['AUTHR_NM'].tolist(),
                    book_data['STORY'].tolist(),
                    "all books"
                )

                # 유사도 계산
                similarities = F.cosine_similarity(
                    all_embeddings,
                    user_embedding.unsqueeze(0)
                )

                # 새로운 추천 도서 선정
                book_data['similarity'] = similarities.cpu().numpy()
                new_recommendations = (
                    book_data[~book_data['ISBN_THIRTEEN_NO'].isin(existing_isbns)]
                    .sort_values('similarity', ascending=False)
                    .head(100)
                    .to_dict('records')
                )

                # 선호도 데이터 로드
                conn = psycopg2.connect(
                    host=os.getenv('DB_HOST'),
                    database=os.getenv('DB_NAME'),
                    user=os.getenv('DB_USER'),
                    password=os.getenv('DB_PASSWORD'),
                    port=os.getenv('DB_PORT')
                )
                query = "SELECT isbn, preference FROM user_preferences WHERE nickname = %s;"
                preferences_df = pd.read_sql(query, conn, params=(nickname,))
                conn.close()

                # 선호도에 따른 유사도 조정
                for _, row in preferences_df.iterrows():
                    isbn = row['isbn']
                    preference = row['preference']
                    book_category = next(
                        (book['category_b'] for book in new_recommendations
                         if book['ISBN_THIRTEEN_NO'] == isbn),
                        None
                    )

                    if book_category:
                        for book in new_recommendations:
                            if book['category_b'] == book_category:
                                if preference == 'like':
                                    book['similarity'] *= 1.2  # 유사도 20% 증가
                                elif preference == 'dislike':
                                    book['similarity'] *= 0.8  # 유사도 20% 감소

                # 새로운 추천 목록 정렬 및 추가
                new_recommendations.sort(key=lambda x: x['similarity'], reverse=True)
                all_recommendations.extend(new_recommendations)
                recommendations_cache[nickname] = all_recommendations

        # 요청된 범위의 도서 반환
        next_books = all_recommendations[offset:offset+10]
        return jsonify({
            'books': next_books,
            'has_more': offset + 10 < len(all_recommendations)
        })

    except Exception as e:
        print(f"Error in load_more_books: {str(e)}")
        return jsonify({'error': str(e)}), 500

@app.route('/get_user_analysis', methods=['GET'])
def get_user_analysis():
    """
    사용자의 도서 선호도 분석 결과를 반환하는 엔드포인트

    Returns:
        JSON 응답:
        - success: 분석 성공 여부
        - analysis: 분석 결과 데이터
            - overall_distribution: 카테고리별 분포
            - favorite_categories: 선호 카테고리
            - disliked_categories: 비선호 카테고리
            - preference_scores: 카테고리별 선호도 점수
    """
    try:
        nickname = request.args.get('nickname')
        analysis = analyze_user_preferences(nickname)
        return jsonify({
            'success': True,
            'analysis': analysis
        })
    except Exception as e:
        return jsonify({
            'success': False,
            'error': str(e)
        }), 500

# ===== 8. 메인 실행 =====
if __name__ == '__main__':
    book_data = fetch_data_from_rds()  # 초기 데이터 로드
    public_url = ngrok.connect(5000)
    print(f"ngrok URL: {public_url}")
    app.run(threaded=True)